In [ ]:
import urllib
import gromacs as gmx
import nglview as nv
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt 

# Preparation

In [ ]:
# Download the PDB file and remove water molecules
pdb='6WX6' 
urllib.request.urlretrieve(f'https://files.rcsb.org/download/{pdb}.pdb',f'{pdb}.pdb')
! grep -v HOH {pdb}.pdb > {pdb}_clean.pdb
pdb = f'{pdb}_clean'

In [ ]:
# Inicialize MDrunnerK8s class for running MD simulation
mdrun=gmx.MDrunnerK8s(image='cerit.io/ljocha/gromacs:2023-2-plumed-2-9-afed-pytorch-model-cv')

def run(mpi=1,omp=2,gpus=1,**kwargs):
    """
    Run a GROMACS molecular dynamics simulation.
    - mpi (int): Number of MPI processes for parallelization.
    - omp (int): Number of OpenMP threads per MPI process.
    - gpus (int): Number of GPUs to use for acceleration.
    - **kwargs: Additional keyword argument.
    """
    mdrun.run(pre={'cores':omp*mpi,'mpi':mpi,'omp':omp,'gpus':gpus}, mdrunargs={**kwargs,'ntomp':omp},ncores=mpi)

In [ ]:
# visualize the molecular structure
nv.show_file(f'{pdb}.pdb')

# Generating topology

In [ ]:
# Generate GROMACS-formatted structure file that contains all the atoms defined within the force field (coordinate file .gro) and topology file (.top)
# Chosed force field: AMBER99
# Chosed water model: TIP3P
gmx.pdb2gmx(f=pdb+'.pdb',o=pdb+'.gro',water='tip3p',ff='amber99',p=pdb+'.top',ignh=True)

# Defining box and Solvating

In [ ]:
# Create a cubic box around the protein structure
# Chosen box type: dodecahedron
# Chosen distance between protein and the edge of the box (at least): 1.5
gmx.editconf(f=pdb+'.gro',o=pdb+'-box.gro',c=True,d='1.5', bt='dodecahedron')

In [ ]:
# Solvate the system by adding water molecules (fill the box with water)
gmx.solvate(cp=pdb+'-box.gro',cs='spc216.gro',o=pdb+'-solv.gro',p=pdb+'.top')

# Adding ions

In [ ]:
# Generate a parameter file (ions.mdp) for adding ions to neutralize the system
# Changed coulombtype=cutoff to coulombtype=PME (as recommended in AMBER manuals)
with open('ions.mdp','w') as ions:
    ions.write("""\
; ions.mdp - used as input into grompp to generate ions.tpr
; Parameters describing what to do, when to stop and what to save
integrator      = steep     ; Algorithm (steep = steepest descent minimization)
emtol           = 1000.0    ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep          = 0.01      ; Minimization step size
nsteps          = 50000     ; Maximum number of (minimization) steps to perform

; Parameters describing how to find the neighbors of each atom and how to calculate the interactions
nstlist         = 1         ; Frequency to update the neighbor list and long range forces
cutoff-scheme	= Verlet    ; Buffered neighbor searching 
ns_type         = grid      ; Method to determine neighbor list (simple, grid)
coulombtype     = PME       ; Treatment of long range electrostatic interactions
rcoulomb        = 1.0       ; Short-range electrostatic cut-off
rvdw            = 1.0       ; Short-range Van der Waals cut-off
pbc             = xyz       ; Periodic Boundary Conditions in all 3 dimensions
""")

# Prepare the ion addition simulation, create .tpr file for genion
gmx.grompp(f='ions.mdp',c=pdb+'-solv.gro',p=pdb+'.top',o='ions.tpr')

In [ ]:
# Create an index file for selecting solvent molecules
gmx.select(s=pdb+'-solv.gro',on='solv.ndx',select='SOL')

In [ ]:
# Add ions to neutralize the system
gmx.genion(s='ions.tpr',n='solv.ndx',o=pdb+'-ions.gro',p=pdb+'.top',pname='NA',nname='CL',neutral=True)

# Energy minimization

In [ ]:
# Generate a parameter file (minim.mdp) for energy minimization
# Changed emstep=0.01 to emstep=0.005 (as in final gmx-demo)
with open('minim.mdp','w') as m:
    m.write("""\
; minim.mdp - used as input into grompp to generate em.tpr
; Parameters describing what to do, when to stop and what to save
integrator      = steep     ; Algorithm (steep = steepest descent minimization)
emtol           = 1000.0    ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep          = 0.005     ; Minimization step size
nsteps          = 50000     ; Maximum number of (minimization) steps to perform

; Parameters describing how to find the neighbors of each atom and how to calculate the interactions
nstlist         = 1         ; Frequency to update the neighbor list and long range forces
cutoff-scheme   = Verlet    ; Buffered neighbor searching
ns_type         = grid      ; Method to determine neighbor list (simple, grid)
coulombtype     = PME       ; Treatment of long range electrostatic interactions
rcoulomb        = 1.0       ; Short-range electrostatic cut-off
rvdw            = 1.0       ; Short-range Van der Waals cut-off
pbc             = xyz       ; Periodic Boundary Conditions in all 3 dimensions

nstxout         = 50         
nstvout         = 0        
nstfout         = 0
nstenergy       = 50         
""")

# Prepare the energy minimization simulation, create .tpr file for mdrun
gmx.grompp(f='minim.mdp',c=pdb+'-ions.gro',p=pdb+'.top',o='em.tpr')

In [ ]:
# Execute the minimization using GROMACS MD engine
# generated files - em.log: ASCII-text log file of the EM process; em.edr: Binary energy file; em.trr: Binary full-precision trajectory; em.gro: Energy-minimized structure
run(deffnm='em')

In [ ]:
# Analyze energy terms
energy = gmx.energy(f='em.edr', o='potential.xvg', terms=[10])

In [ ]:
# Create an index file (protein.ndx) containing only atoms that belong to the protein 
gmx.select(s=pdb+'-ions.gro',on='protein.ndx',select='Protein')

In [ ]:
# Convert the trajectory file to XTC format
gmx.trjconv(s=pdb+'-ions.gro',f='em.trr',n='protein.ndx',o='em-protein.xtc')

In [ ]:
# Load the trajectory file and visualize it using NGLView
tr=md.load('em-protein.xtc',top=pdb+'.gro')
nv.show_mdtraj(tr)

# Equilibration

In [ ]:
# Generate a parameter file (nvt.mdp) for NVT equilibration
with open('nvt.mdp','w') as nvt:
    nvt.write("""\
title                   = OPLS Lysozyme NVT equilibration 
define                  = -DPOSRES  ; Position restrain the protein              
; nvt.mdp - used as input into grompp to generate nvt.tpr
; Run parameters
integrator              = md        ; Algorithm (md = molecular dynamics)
nsteps                  = 50000     ; Maximum number of steps to perform
dt                      = 0.002     ; Time step (in ps)
              
; Bond parameters
continuation            = no        ; First dynamics run
constraint_algorithm    = lincs     ; Holonomic constraints 
constraints             = h-bonds   ; Bonds involving H are constrained
lincs_iter              = 1         ; Accuracy of LINCS
lincs_order             = 4         ; Also related to accuracy              
              
; Nonbonded settings 
cutoff-scheme           = Verlet    ; Buffered neighbor searching
ns_type                 = grid      ; Search neighboring grid cells
nstlist                 = 10        ; 20 fs, largely irrelevant with Verlet
rcoulomb                = 1.0       ; Short-range electrostatic cutoff (in nm)
rvdw                    = 1.0       ; Short-range van der Waals cutoff (in nm)
DispCorr                = EnerPres  ; Account for cut-off vdW scheme
              
; Electrostatics
coulombtype             = PME       ; Particle Mesh Ewald for long-range electrostatics
pme_order               = 4         ; cubic interpolation
fourierspacing          = 0.16      ; Grid spacing for FFT              

; Periodic boundary conditions
pbc                     = xyz       ; 3-D PBC

; Temperature coupling
tcoupl                  = V-rescale             ; Temperature coupling type
tc-grps                 = Protein Non-Protein   ; Two coupling groups - more accurate
tau_t                   = 0.1     0.1           ; Time constant, in ps
ref_t                   = 300     300           ; Reference temperature, one for each group, in K
              
; Pressure coupling
pcoupl                  = no        ; No pressure coupling in NVT              

; Velocity generation
gen_vel                 = yes       ; Generate velocities at start
gen_temp                = 300       ; Initial temperature (in K)
gen_seed                = -1        ; Random seed for velocity generation

; Output control
nstxout                 = 500       ; Save coordinates every 1000 steps
nstvout                 = 500       ; Save velocities every 1000 steps
nstenergy               = 500       ; Save energies every 1000 steps
nstlog                  = 500       ; Update log file every 1.0 ps              
""")
    
# Prepare the NVT equilibration simulation, create .tpr file for mdrun
gmx.grompp(f='nvt.mdp',c='em.gro',r='em.gro',p=pdb+'.top',o='nvt.tpr')

In [ ]:
# Execute the NVT equilibration using GROMACS MD engine
run(deffnm='nvt')

In [ ]:
# Plot
temp = np.loadtxt('temp.xvg',comments=['#','@'])
press = np.loadtxt('press.xvg',comments=['#','@'])

plt.figure(figsize=(15,9))
plt.subplot(211)
plt.plot(press[:,0],press[:,1])
plt.title('isothermal-isochoric equilibration')
plt.grid()
#plt.xlabel('time (ps)')
plt.ylabel("pressure (bar)")


plt.subplot(212)
plt.xlabel('time (ps)')
plt.ylabel('temperature (K)')
plt.grid()
plt.plot(temp[:,0],temp[:,1])

plt.show()

In [ ]:
# Generate a parameter file (npt.mdp) for NPT equilibration   
with open('npt.mdp','w') as nvt:
    nvt.write("""\
define                  = -DPOSRES  ; Position restrain the protein              
; Run parameters
integrator              = md        ; Algorithm (md = molecular dynamics)
nsteps                  = 50000     ; Maximum number of steps to perform
dt                      = 0.002     ; Time step (in ps)

; Bond parameters
continuation            = yes       ; Restarting after NVT 
constraint_algorithm    = lincs     ; Holonomic constraints 
constraints             = h-bonds   ; Bonds involving H are constrained
lincs_iter              = 1         ; Accuracy of LINCS
lincs_order             = 4         ; Also related to accuracy
                                          
; Nonbonded settings 
cutoff-scheme           = Verlet    ; Buffered neighbor searching
ns_type                 = grid      ; Search neighboring grid cells
nstlist                 = 10        ; 20 fs, largely irrelevant with Verlet
rcoulomb                = 1.0       ; Short-range electrostatic cutoff (in nm)
rvdw                    = 1.0       ; Short-range van der Waals cutoff (in nm)
DispCorr                = EnerPres  ; Account for cut-off vdW scheme
              
; Electrostatics
coulombtype             = PME       ; Particle Mesh Ewald for long-range electrostatics
pme_order               = 4         ; Cubic interpolation
fourierspacing          = 0.16      ; Grid spacing for FFT              
                                  
; Periodic boundary conditions
pbc                     = xyz       ; 3-D PBC

; Temperature coupling
tcoupl                  = V-rescale             ; Temperature coupling type
tc-grps                 = Protein Non-Protein   ; Two coupling groups - more accurate
tau_t                   = 0.1     0.1           ; Time constant, in ps
ref_t                   = 300     300           ; Reference temperature, one for each group, in K
              
; Pressure coupling
; ljocha pcoupl                  = Parrinello-Rahman     ; Pressure coupling on in NPT
pcoupl                  = C-rescale
pcoupltype              = isotropic  ; uniform scaling of box vectors         
; ljocha tau_p                   = 2.0                   ; Time constant, in ps
tau_p                   = 5.0
ref_p                   = 1.0       ; Reference pressure, in bar
compressibility         = 4.5e-5    ; Isothermal compressibility of water, bar^-1
refcoord_scaling        = com
              
; Velocity generation
gen_vel                 = no        ; Velocity generation is off 

; Output control
nstxout                 = 500       ; Save coordinates every 1000 steps
nstvout                 = 500       ; Save velocities every 1000 steps
nstenergy               = 500       ; Save energies every 1000 steps
nstlog                  = 500       ; Update log file every 1.0 ps              
""")

# Prepare the NPT equilibration simulation, create .tpr file for mdrun
gmx.grompp(f='npt.mdp', c='nvt.gro', r='nvt.gro', p=pdb+'.top', o='npt.tpr')

In [ ]:
# Execute the NPT equilibration using GROMACS MD engine
run(deffnm='npt')

In [ ]:
# Analyze pressure progression
gmx.energy(f='npt.edr',o='press.xvg',input='Pressure')

# Analyze density
gmx.energy(f='npt.edr',o='dens.xvg',input='Density')

# Analyze temperature progression
gmx.energy(f='npt.edr',o='temp.xvg',input='Temperature')

In [ ]:
# Plot
temp = np.loadtxt('temp.xvg',comments=['#','@'])
press = np.loadtxt('press.xvg',comments=['#','@'])
dens = np.loadtxt('dens.xvg',comments=['#','@'])

plt.figure(figsize=(15,9))
plt.subplot(311)
plt.plot(press[:,0],press[:,1])
plt.title('isothermal-isobaric equilibration')
plt.grid()
#plt.xlabel('time (ps)')
plt.ylabel("pressure (bar)")

plt.subplot(312)
plt.ylabel('density (kg/m3)')
plt.grid()
plt.plot(dens[:,0],dens[:,1])

plt.subplot(313)
plt.xlabel('time (ps)')
plt.ylabel('temperature (K)')
plt.grid()
plt.plot(temp[:,0],temp[:,1])

plt.show()

# Production MD

In [ ]:
time = 100 #ps
nsteps = time * 500 # XXX: dt = 0.002

# Generate a parameter file (md.mdp) for MD simulation 
with open('md.mdp','w') as mdp:
    mdp.write(f"""\
integrator              = md        ; Leap-frog integrator
dt                      = 0.002     ; 2 fs

; Bond parameters
continuation            = yes       ; Restarting after NPT 
constraint_algorithm    = lincs     ; Holonomic constraints 
constraints             = h-bonds   ; Bonds involving H are constrained
lincs_iter              = 1         ; Accuracy of LINCS
lincs_order             = 4         ; Also related to accuracy
              
; Nonbonded settings
cutoff-scheme           = Verlet    ; Buffered neighbor searching
ns_type                 = grid      ; Search neighboring grid cells
nstlist                 = 10        ; 20 fs, largely irrelevant with Verlet scheme
rcoulomb                = 1.0       ; Short-range electrostatic cutoff (in nm)
rvdw                    = 1.0       ; Short-range van der Waals cutoff (in nm)
              
; Electrostatics
coulombtype             = PME       ; Particle Mesh Ewald for long-range electrostatics
pme_order               = 4         ; Cubic interpolation
fourierspacing          = 0.16      ; Grid spacing for FFT
              
; Periodic boundary conditions
pbc                     = xyz       ; 3-D PBC              
              
; Temperature coupling
tcoupl                  = V-rescale             ; Temperature coupling type
tc-grps                 = Protein Non-Protein   ; Two coupling groups - more accurate
tau_t                   = 0.1     0.1           ; Time constant, in ps
ref_t                   = 300     300           ; Reference temperature, one for each group, in K
              
; Pressure coupling
pcoupl                  = Parrinello-Rahman     ; Pressure coupling on in NPT
pcoupltype              = isotropic             ; uniform scaling of box vectors
tau_p                   = 2.0                   ; time constant, in ps
ref_p                   = 1.0                   ; reference pressure, in bar
compressibility         = 4.5e-5                ; isothermal compressibility of water, bar^-1
              
; Dispersion correction
DispCorr                = EnerPres  ; account for cut-off vdW scheme
              
; Velocity generation
gen_vel                 = no        ; Velocity generation is off 
nsteps = {nsteps}

; Output control
nstxout                 = 0         ; Suppress bulky .trr file by specifying 
nstvout                 = 0         ; 0 for output frequency of nstxout,
nstfout                 = 0         ; nstvout, and nstfout
nstenergy               = 5000      ; Save energies every 10.0 ps
nstlog                  = 5000      ; Update log file every 10.0 ps
nstxout-compressed      = 5000      ; Save compressed coordinates every 10.0 ps
compressed-x-grps       = System    ; Save the whole system
""")    


# Prepare the MD simulation, create .tpr file for mdrun
#gmx.grompp(f='md.mdp', c='npt.gro', t='npt.cpt', p=pdb+'.top', o='md_0_1.tpr')
gmx.grompp(f="md.mdp",c="npt.gro",r="npt.gro",p=pdb+'.top',o="md.tpr")
# gmx.grompp(f="md.mdp",c="npt.gro",r="npt.gro",p=pdb+'.top',o="md.tpr")

In [ ]:
# Execute the MD simulation
run(deffnm='md')

# Analysis

In [ ]:
# Create an index file (protein.ndx) containing only atoms that belong to the protein
# gmx.select(s=pdb+'-ions.gro',on='protein.ndx',select='Protein')
gmx.select(s='md_0_1.gro',on='protein.ndx',select='Protein') 

In [ ]:
# Convert the trajectory file to XTC format
# gmx.trjconv(s=pdb+'-ions.gro',f='em.trr',n='protein.ndx',o='em-protein.xtc')

In [ ]:
# Convert the trajectory file to XTC format
# gmx.trjconv(s=pdb+'-ions.gro',f='md_0_1.trr',n='protein.ndx',o='em-protein.xtc')
gmx.trjconv(s='md_0_1.gro',f='md_0_1.trr',n='protein.ndx',o='md_0_1.xtc')

In [ ]:
# Process trajectory to remove the effects of periodic boundary conditions
gmx.trjconv(s='md_0_1.tpr', f='md_0_1.xtc', o='md_0_1_noPBC.xtc', pbc='mol', center=True, group=1, ogroup=0)

In [ ]:
# Calculate RMSD
gmx.rms(s='md_0_1.tpr', f='md_0_1_noPBC.xtc', o='rmsd.xvg', tu='ns', group=4, fitgroup=4)

In [ ]:
# Calculate RMSD relative to the crystal structure
gmx.rms(s='em.tpr', f='md_0_1_noPBC.xtc', o='rmsd_xtal.xvg', tu='ns', group=4, fitgroup=4)

In [ ]:
# Run gyrate analysis
gmx.gyrate(s='md_0_1.tpr', f='md_0_1_noPBC.xtc', o='gyrate.xvg')

# Visualization

In [ ]:
# Load the trajectory file and visualize it using NGLView
# tr=md.load('em-protein.xtc',top=pdb+'.gro')
tr=md.load('md_0_1.xtc',top=pdb+'.top')

nv.show_mdtraj(tr)